# Merge a dataset through 2020 with audio feature metadata (importantly the genre), pull lyrics from dataset with pre-pulled lyrics through 2019, snag 2020 lyrics via genius API

In [1]:
import pandas as pd
import numpy as np

In [3]:
np.__version__

'1.19.5'

## Load the data through 2020, merge with genres
[data source](https://data.world/kcmillersean/billboard-hot-100-1958-2017/workspace/data-dictionary)

In [53]:
new_song_df = pd.read_csv('./data/Hot Stuff.csv')

In [58]:
new_song_df.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4
1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5
2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6
3,http://www.billboard.com/charts/hot-100/1965-0...,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7
4,http://www.billboard.com/charts/hot-100/1965-0...,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8


In [54]:
audio_features = pd.read_csv('./data/Hot 100 Audio Features.csv')

In [57]:
audio_features

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,......And Roses And RosesAndy Williams,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",3tvqPPpXyIgKrm4PR9HCf0,https://p.scdn.co/mp3-preview/cef4883cfd1e0e53...,166106.0,False,The Essential Andy Williams,0.154,...,-14.063,1.0,0.0315,0.91100,0.000267,0.1120,0.150,83.969,4.0,38.0
3,...And Then There Were DrumsSandy Nelson,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",1fHHq3qHU8wpRKHzhojZ4a,NaN,172066.0,False,Compelling Percussion,0.588,...,-17.278,0.0,0.0361,0.00256,0.745000,0.1450,0.801,121.962,4.0,11.0
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",3MjUtNVVq3C8Fn0MP3zhXa,https://p.scdn.co/mp3-preview/da2134a161f1cb34...,211066.0,False,...Baby One More Time (Digital Deluxe Version),0.759,...,-5.745,0.0,0.0307,0.20200,0.000131,0.4430,0.907,92.960,4.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29498,Zoo YorkLil Tjay Featuring Fivio Foreign & Pop...,Lil Tjay Featuring Fivio Foreign & Pop Smoke,Zoo York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29499,ZoomFuture,Future,Zoom,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",2IG6Te7JyvrtqhFeOF7le4,https://p.scdn.co/mp3-preview/cb8fde6edc08e70a...,278429.0,True,FUTURE,0.852,...,-7.673,1.0,0.4260,0.01450,0.000000,0.2630,0.627,150.945,4.0,51.0
29500,ZoomLil' Boosie Featuring Yung Joc,Lil' Boosie Featuring Yung Joc,Zoom,"['baton rouge rap', 'deep southern trap']",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29501,Zorba The GreekHerb Alpert & The Tijuana Brass,Herb Alpert & The Tijuana Brass,Zorba The Greek,"['adult standards', 'easy listening', 'lounge']",3WLEVNohakzZmMpN5W7mHK,https://p.scdn.co/mp3-preview/1841a4034ba42fc0...,264853.0,False,!!!Going Places!!!,0.531,...,-12.702,1.0,0.3230,0.15400,0.279000,0.0584,0.192,82.107,4.0,35.0


# NOTE: I'm just going with songs that breached the top 20 at least once to narrow the scope, for now 
(I may want to to expand this later)

In [69]:
# Only keep songs that hit top 20
temp = new_song_df.sort_values('Peak Position').groupby(['SongID']).first()
only_top_20 = temp[temp['Peak Position'] <= 20].reset_index()
only_top_20

,SongID,url,WeekID,Week Position,Song,Performer,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,#9 DreamJohn Lennon,http://www.billboard.com/charts/hot-100/1975-0...,2/22/1975,9,#9 Dream,John Lennon,1,10.0,9,10
1,#BeautifulMariah Carey Featuring Miguel,http://www.billboard.com/charts/hot-100/2013-0...,8/3/2013,37,#Beautiful,Mariah Carey Featuring Miguel,1,23.0,15,11
2,#SELFIEThe Chainsmokers,http://www.billboard.com/charts/hot-100/2014-0...,4/26/2014,33,#SELFIE,The Chainsmokers,1,20.0,16,7
3,#thatPOWERwill.i.am Featuring Justin Bieber,http://www.billboard.com/charts/hot-100/2013-0...,6/8/2013,19,#thatPOWER,will.i.am Featuring Justin Bieber,1,19.0,17,10
4,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,http://www.billboard.com/charts/hot-100/2002-1...,12/28/2002,4,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,1,5.0,4,10
...,...,...,...,...,...,...,...,...,...,...
7951,interludeJ. Cole,https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,17,interlude,J. Cole,1,8.0,8,2
7952,"let.go.my.handJ. Cole, Bas & 6LACK",https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,19,let.go.my.hand,"J. Cole, Bas & 6LACK",1,NaN,19,1
7953,"my.lifeJ. Cole, 21 Savage & Morray",https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,2,my.life,"J. Cole, 21 Savage & Morray",1,NaN,2,1
7954,pride.is.the.devilJ. Cole & Lil Baby,https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,7,pride.is.the.devil,J. Cole & Lil Baby,1,NaN,7,1


In [187]:
# merge on songId with audio features
merged_df = (
    only_top_20.merge(left_on='SongID', right=audio_features, right_on='SongID', how='left')
).drop_duplicates(subset=['SongID'])

In [188]:
merged_df

,SongID,url,WeekID,Week Position,Song_x,Performer_x,Instance,Previous Week Position,Peak Position,Weeks on Chart,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,#9 DreamJohn Lennon,http://www.billboard.com/charts/hot-100/1975-0...,2/22/1975,9,#9 Dream,John Lennon,1,10.0,9,10,...,-11.745,1.0,0.0336,0.07440,0.013300,0.2540,0.478,115.474,4.0,55.0
1,#BeautifulMariah Carey Featuring Miguel,http://www.billboard.com/charts/hot-100/2013-0...,8/3/2013,37,#Beautiful,Mariah Carey Featuring Miguel,1,23.0,15,11,...,-5.405,1.0,0.0391,0.34600,0.000000,0.3470,0.469,107.042,4.0,56.0
2,#SELFIEThe Chainsmokers,http://www.billboard.com/charts/hot-100/2014-0...,4/26/2014,33,#SELFIE,The Chainsmokers,1,20.0,16,7,...,-3.282,1.0,0.2260,0.01340,0.000012,0.0787,0.651,127.973,4.0,65.0
3,#thatPOWERwill.i.am Featuring Justin Bieber,http://www.billboard.com/charts/hot-100/2013-0...,6/8/2013,19,#thatPOWER,will.i.am Featuring Justin Bieber,1,19.0,17,10,...,-6.096,0.0,0.0583,0.00112,0.000077,0.0748,0.403,128.000,4.0,68.0
4,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,http://www.billboard.com/charts/hot-100/2002-1...,12/28/2002,4,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,1,5.0,4,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005,interludeJ. Cole,https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,17,interlude,J. Cole,1,8.0,8,2,...,-8.336,1.0,0.4610,0.62600,0.000000,0.1190,0.138,84.066,4.0,57.0
8006,"let.go.my.handJ. Cole, Bas & 6LACK",https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,19,let.go.my.hand,"J. Cole, Bas & 6LACK",1,NaN,19,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8007,"my.lifeJ. Cole, 21 Savage & Morray",https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,2,my.life,"J. Cole, 21 Savage & Morray",1,NaN,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8008,pride.is.the.devilJ. Cole & Lil Baby,https://www.billboard.com/charts/hot-100/2021-...,5/29/2021,7,pride.is.the.devil,J. Cole & Lil Baby,1,NaN,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
USEFUL_COLUMNS = [
    'SongID', 'Performer_x', 'Song_x', 'spotify_genre',
       'spotify_track_preview_url',
       'spotify_track_explicit', 'spotify_track_album', 'WeekID', 'Week Position', 'Peak Position'
]
merged_df = merged_df[USEFUL_COLUMNS].rename(columns={'Performer_x': 'Performer', 'Song_x': 'Song'})
merged_df

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position
0,#9 DreamJohn Lennon,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...",NaN,False,Walls And Bridges,2/22/1975,9,9
1,#BeautifulMariah Carey Featuring Miguel,Mariah Carey Featuring Miguel,#Beautiful,"['dance pop', 'pop', 'r&b', 'urban contemporary']",NaN,False,Me. I Am Mariah…The Elusive Chanteuse (Deluxe),8/3/2013,37,15
2,#SELFIEThe Chainsmokers,The Chainsmokers,#SELFIE,"['electropop', 'pop', 'tropical house']",NaN,False,#SELFIE,4/26/2014,33,16
3,#thatPOWERwill.i.am Featuring Justin Bieber,will.i.am Featuring Justin Bieber,#thatPOWER,"['dance pop', 'pop', 'pop rap', 'post-teen pop']",NaN,False,#willpower (Deluxe),6/8/2013,19,17
4,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,Jay-Z Featuring Beyonce Knowles,'03 Bonnie & Clyde,"['east coast hip hop', 'hip hop', 'pop rap', '...",NaN,True,The Blueprint 2 The Gift & The Curse,12/28/2002,4,4
...,...,...,...,...,...,...,...,...,...,...
8005,interludeJ. Cole,J. Cole,interlude,"['conscious hip hop', 'north carolina hip hop'...",NaN,True,"{'album_type': 'album', 'artists': [{'external...",5/29/2021,17,8
8006,"let.go.my.handJ. Cole, Bas & 6LACK","J. Cole, Bas & 6LACK",let.go.my.hand,NaN,NaN,NaN,NaN,5/29/2021,19,19
8007,"my.lifeJ. Cole, 21 Savage & Morray","J. Cole, 21 Savage & Morray",my.life,NaN,NaN,NaN,NaN,5/29/2021,2,2
8008,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,NaN,NaN,NaN,NaN,5/29/2021,7,7


## Parse genre

In [129]:
# TODO if needed

## Attach lyrics
[Start by just merging the lyrics scraped through 2019 by this person](https://towardsdatascience.com/sentiment-analysis-of-all-billboard-hot-100-songs-over-time-1958-2019-3329439e7c1a)


In [84]:
pre_scraped_lyrics_through_2019 = pd.read_csv('./data/lyric-full_df.csv')[['SongID', 'Lyrics']]

In [191]:
merged_with_thru_2019_lyrics = merged_df.merge(left_on='SongID', right=pre_scraped_lyrics_through_2019, right_on='SongID', how='left')
merged_with_thru_2019_lyrics

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position,Lyrics
0,#9 DreamJohn Lennon,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...",NaN,False,Walls And Bridges,2/22/1975,9,9,"[Verse 1]\nSo long ago\nWas it in a dream, was..."
1,#BeautifulMariah Carey Featuring Miguel,Mariah Carey Featuring Miguel,#Beautiful,"['dance pop', 'pop', 'r&b', 'urban contemporary']",NaN,False,Me. I Am Mariah…The Elusive Chanteuse (Deluxe),8/3/2013,37,15,"[Intro: Mariah Carey]\nAh, ah, youâre beauti..."
2,#SELFIEThe Chainsmokers,The Chainsmokers,#SELFIE,"['electropop', 'pop', 'tropical house']",NaN,False,#SELFIE,4/26/2014,33,16,[Produced by The Chainsmokers]\n\n[Verse 1]\nW...
3,#thatPOWERwill.i.am Featuring Justin Bieber,will.i.am Featuring Justin Bieber,#thatPOWER,"['dance pop', 'pop', 'pop rap', 'post-teen pop']",NaN,False,#willpower (Deluxe),6/8/2013,19,17,To help celebrate summer Pepsi compiled a list...
4,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,Jay-Z Featuring Beyonce Knowles,'03 Bonnie & Clyde,"['east coast hip hop', 'hip hop', 'pop rap', '...",NaN,True,The Blueprint 2 The Gift & The Curse,12/28/2002,4,4,"[Intro: Jay Z]\nUhh, uhh, uhh\nYou ready, B?\n..."
...,...,...,...,...,...,...,...,...,...,...,...
7951,interludeJ. Cole,J. Cole,interlude,"['conscious hip hop', 'north carolina hip hop'...",NaN,True,"{'album_type': 'album', 'artists': [{'external...",5/29/2021,17,8,NaN
7952,"let.go.my.handJ. Cole, Bas & 6LACK","J. Cole, Bas & 6LACK",let.go.my.hand,NaN,NaN,NaN,NaN,5/29/2021,19,19,NaN
7953,"my.lifeJ. Cole, 21 Savage & Morray","J. Cole, 21 Savage & Morray",my.life,NaN,NaN,NaN,NaN,5/29/2021,2,2,NaN
7954,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,NaN,NaN,NaN,NaN,5/29/2021,7,7,NaN


### Scrape Lyrics for 2020 from genius.com API

In [87]:
import lyricsgenius

In [90]:
GENIUS_CLIEN_API_KEY = '8ZiCNVQqbSF6NM0nwVnVv6Y0EBR7BsVQ9HaVgPdqJOZR1l6XlymR-pUUi6z3GHPx' # be nice :)
genius = lyricsgenius.Genius(GENIUS_CLIEN_API_KEY)

In [112]:
def get_lyrics(title='', artist=''):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        print('not found: ${title}')

In [192]:
tiny = merged_with_thru_2019_lyrics[
    merged_with_thru_2019_lyrics['Lyrics'].isna()
]
songs_without_lyrics_from_2020 = tiny[
    tiny['WeekID'].str.endswith('/2020') == True
]

In [193]:
songs_without_lyrics_from_2020

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position,Lyrics
87,"10,000 HoursDan + Shay & Justin Bieber",Dan + Shay & Justin Bieber,"10,000 Hours","['contemporary country', 'pop']",NaN,NaN,NaN,1/4/2020,17,4,NaN
111,34+35Ariana Grande,Ariana Grande,34+35,"['dance pop', 'pop', 'post-teen pop']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",12/5/2020,30,8,NaN
128,7 SummersMorgan Wallen,Morgan Wallen,7 Summers,['contemporary country'],NaN,False,"{'album_type': 'album', 'artists': [{'external...",12/12/2020,76,6,NaN
225,Adore YouHarry Styles,Harry Styles,Adore You,"['dance pop', 'pop', 'post-teen pop']",https://p.scdn.co/mp3-preview/2dbeaa922270b337...,False,Adore You,5/30/2020,12,6,NaN
232,After HoursThe Weeknd,The Weeknd,After Hours,"['canadian contemporary r&b', 'canadian pop', ...",NaN,False,"{'album_type': 'album', 'artists': [{'external...",3/21/2020,100,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7452,"Whats PoppinJack Harlow Featuring DaBaby, Tory...","Jack Harlow Featuring DaBaby, Tory Lanez & Lil...",Whats Poppin,NaN,NaN,NaN,NaN,8/15/2020,3,2,NaN
7635,Wishing WellJuice WRLD,Juice WRLD,Wishing Well,"['chicago rap', 'melodic rap']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",8/22/2020,27,5,NaN
7660,WoahLil Baby,Lil Baby,Woah,"['atl hip hop', 'atl trap', 'pop rap', 'rap', ...",NaN,True,Woah,3/21/2020,31,15,NaN
7672,WonderShawn Mendes,Shawn Mendes,Wonder,"['canadian pop', 'dance pop', 'pop', 'post-tee...",NaN,False,"{'album_type': 'single', 'artists': [{'externa...",12/19/2020,70,18,NaN


## Do the actual Genius API calls (I stashed results to disk)

In [194]:
songs_without_lyrics_from_2020

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position,Lyrics
87,"10,000 HoursDan + Shay & Justin Bieber",Dan + Shay & Justin Bieber,"10,000 Hours","['contemporary country', 'pop']",NaN,NaN,NaN,1/4/2020,17,4,NaN
111,34+35Ariana Grande,Ariana Grande,34+35,"['dance pop', 'pop', 'post-teen pop']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",12/5/2020,30,8,NaN
128,7 SummersMorgan Wallen,Morgan Wallen,7 Summers,['contemporary country'],NaN,False,"{'album_type': 'album', 'artists': [{'external...",12/12/2020,76,6,NaN
225,Adore YouHarry Styles,Harry Styles,Adore You,"['dance pop', 'pop', 'post-teen pop']",https://p.scdn.co/mp3-preview/2dbeaa922270b337...,False,Adore You,5/30/2020,12,6,NaN
232,After HoursThe Weeknd,The Weeknd,After Hours,"['canadian contemporary r&b', 'canadian pop', ...",NaN,False,"{'album_type': 'album', 'artists': [{'external...",3/21/2020,100,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7452,"Whats PoppinJack Harlow Featuring DaBaby, Tory...","Jack Harlow Featuring DaBaby, Tory Lanez & Lil...",Whats Poppin,NaN,NaN,NaN,NaN,8/15/2020,3,2,NaN
7635,Wishing WellJuice WRLD,Juice WRLD,Wishing Well,"['chicago rap', 'melodic rap']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",8/22/2020,27,5,NaN
7660,WoahLil Baby,Lil Baby,Woah,"['atl hip hop', 'atl trap', 'pop rap', 'rap', ...",NaN,True,Woah,3/21/2020,31,15,NaN
7672,WonderShawn Mendes,Shawn Mendes,Wonder,"['canadian pop', 'dance pop', 'pop', 'post-tee...",NaN,False,"{'album_type': 'single', 'artists': [{'externa...",12/19/2020,70,18,NaN


In [197]:
# get_lyrics(title='Adore You', artist='Harry Styles')
# get_lyrics(title='Adore You', artist='Dan + Shay & Justin Bieber')
lyrics_for_2020_songs = songs_without_lyrics_from_2020.apply(
    lambda row: get_lyrics(title=row['Song'], artist=row['Performer']),
    axis=1
)

Searching for "10,000 Hours" by Dan + Shay & Justin Bieber...
Done.
Searching for "34+35" by Ariana Grande...
Done.
Searching for "7 Summers" by Morgan Wallen...
Done.
Searching for "Adore You" by Harry Styles...
Done.
Searching for "After Hours" by The Weeknd...
Done.
Searching for "BOP" by DaBaby...
Done.
Searching for "Baby Pluto" by Lil Uzi Vert...
Done.
Searching for "Bad Energy" by Juice WRLD...
Done.
Searching for "Ballin'" by Mustard & Roddy Ricch...
Done.
Searching for "Be Like That" by Kane Brown With Swae Lee & Khalid...
Done.
Searching for "Bean (Kobe)" by Lil Uzi Vert Featuring Chief Keef...
Done.
Searching for "Before You Go" by Lewis Capaldi...
Done.
Searching for "Blinding Lights" by The Weeknd...
Done.
Searching for "Blood On My Jeans" by Juice WRLD...
Done.
Searching for "Blue & Grey" by BTS...
Done.
Searching for "Blueberry Faygo" by Lil Mosey...
Done.
Searching for "Break My Heart" by Dua Lipa...
Done.
Searching for "Cardigan" by Taylor Swift...
Done.
Searching for 

In [ ]:
# pd.DataFrame({'lyrics': lyrics_for_2020_songs}).to_csv('./data/2020 lyrics pulled')

In [202]:
lyrics_for_2020_songs.isna().sum()

1

In [203]:
songs_without_lyrics_from_2020['Lyrics'] = lyrics_for_2020_songs
# songs_without_lyrics_from_2020.to_csv('', index=False)

/Users/dmora/.virtualenvs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [204]:
songs_without_lyrics_from_2020

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position,Lyrics
87,"10,000 HoursDan + Shay & Justin Bieber",Dan + Shay & Justin Bieber,"10,000 Hours","['contemporary country', 'pop']",NaN,NaN,NaN,1/4/2020,17,4,"[Verse 1: Dan + Shay]\nDo you love the rain, d..."
111,34+35Ariana Grande,Ariana Grande,34+35,"['dance pop', 'pop', 'post-teen pop']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",12/5/2020,30,8,[Intro]\nHmm\n\n[Verse 1]\nYou might think I'm...
128,7 SummersMorgan Wallen,Morgan Wallen,7 Summers,['contemporary country'],NaN,False,"{'album_type': 'album', 'artists': [{'external...",12/12/2020,76,6,"[Verse 1]\nYeah, you used to talk about\nGetti..."
225,Adore YouHarry Styles,Harry Styles,Adore You,"['dance pop', 'pop', 'post-teen pop']",https://p.scdn.co/mp3-preview/2dbeaa922270b337...,False,Adore You,5/30/2020,12,6,[Verse 1: Harry Styles]\nWalk in your rainbow ...
232,After HoursThe Weeknd,The Weeknd,After Hours,"['canadian contemporary r&b', 'canadian pop', ...",NaN,False,"{'album_type': 'album', 'artists': [{'external...",3/21/2020,100,20,[Verse 1]\nThought I almost died in my dream a...
...,...,...,...,...,...,...,...,...,...,...,...
7452,"Whats PoppinJack Harlow Featuring DaBaby, Tory...","Jack Harlow Featuring DaBaby, Tory Lanez & Lil...",Whats Poppin,NaN,NaN,NaN,NaN,8/15/2020,3,2,[Verse 1: Jack Harlow]\nTornato con il remix\n...
7635,Wishing WellJuice WRLD,Juice WRLD,Wishing Well,"['chicago rap', 'melodic rap']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",8/22/2020,27,5,"[Intro]\nMm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI c..."
7660,WoahLil Baby,Lil Baby,Woah,"['atl hip hop', 'atl trap', 'pop rap', 'rap', ...",NaN,True,Woah,3/21/2020,31,15,[Intro]\nNew car very noisy\nCome through and ...
7672,WonderShawn Mendes,Shawn Mendes,Wonder,"['canadian pop', 'dance pop', 'pop', 'post-tee...",NaN,False,"{'album_type': 'single', 'artists': [{'externa...",12/19/2020,70,18,[Verse 1]\nI wonder if I'm being real\nDo I sp...


#### Merge back in the updated 2020 songs now with lyrics

In [205]:
merged_df_minus_updated = merged_with_thru_2019_lyrics[
    ~merged_with_thru_2019_lyrics.index.isin(songs_without_lyrics_from_2020.index)
]
fully_updated_df = pd.concat([merged_df_minus_updated, songs_without_lyrics_from_2020])

In [210]:
print(f'With lyrics: {fully_updated_df.Lyrics.notna().sum()}\nMissing lyrics: {fully_updated_df.Lyrics.isna().sum()}')

With lyrics: 7483
Missing lyrics: 473


## Clean up the now fully merged data

In [167]:
# drop 
ready_for_export = fully_updated_df.dropna(subset=['Lyrics','WeekID'])
ready_for_export

,SongID,Performer,Song,spotify_genre,spotify_track_preview_url,spotify_track_explicit,spotify_track_album,WeekID,Week Position,Peak Position,Lyrics
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",https://p.scdn.co/mp3-preview/da2134a161f1cb34...,False,...Baby One More Time (Digital Deluxe Version),5/29/1999,31.0,1.0,"[Intro]\nOh baby, baby\nOh baby, baby\n\n[Vers..."
5,...Ready For It?Taylor Swift,Taylor Swift,...Ready For It?,"['pop', 'post-teen pop']",NaN,False,"{'album_type': 'album', 'artists': [{'external...",12/2/2017,18.0,4.0,[Verse 1]\nKnew he was a killer first time tha...
6,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,Jay-Z Featuring Beyonce Knowles,'03 Bonnie & Clyde,"['east coast hip hop', 'hip hop', 'pop rap', '...",NaN,True,The Blueprint 2 The Gift & The Curse,12/28/2002,4.0,4.0,"[Intro: Jay Z]\nUhh, uhh, uhh\nYou ready, B?\n..."
7,'65 Love AffairPaul Davis,Paul Davis,'65 Love Affair,"['album rock', 'bubblegum pop', 'country rock'...",https://p.scdn.co/mp3-preview/a701445830ecacfb...,False,Radio Hits Of the '80s,6/12/1982,18.0,6.0,'65 love affair\nPaul Davis\n\nI was a car hop...
15,'TilThe Angels,The Angels,'Til,"['blues rock', 'garage rock', 'modern blues ro...",https://p.scdn.co/mp3-preview/bf264f14124ddf08...,False,Caprice! The Story Behind Gerry Granahan's Cap...,1/20/1962,38.0,14.0,"[Intro]\nYeah, yeah, yeah\nRap diablo\nYeah, y..."
...,...,...,...,...,...,...,...,...,...,...,...
27647,"Whats PoppinJack Harlow Featuring DaBaby, Tory...","Jack Harlow Featuring DaBaby, Tory Lanez & Lil...",Whats Poppin,NaN,NaN,NaN,NaN,8/15/2020,3.0,2.0,[Verse 1: Jack Harlow]\nTornato con il remix\n...
28351,Wishing WellJuice WRLD,Juice WRLD,Wishing Well,"['chicago rap', 'melodic rap']",NaN,True,"{'album_type': 'album', 'artists': [{'external...",8/22/2020,27.0,5.0,"[Intro]\nMm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI c..."
28427,WoahLil Baby,Lil Baby,Woah,"['atl hip hop', 'atl trap', 'pop rap', 'rap', ...",NaN,True,Woah,3/21/2020,31.0,15.0,[Intro]\nNew car very noisy\nCome through and ...
28497,WonderShawn Mendes,Shawn Mendes,Wonder,"['canadian pop', 'dance pop', 'pop', 'post-tee...",NaN,False,"{'album_type': 'single', 'artists': [{'externa...",12/19/2020,70.0,18.0,[Verse 1]\nI wonder if I'm being real\nDo I sp...


In [214]:
ready_for_export.to_csv('./data/Songs with metadata and lyrics.csv', index=False)